# Train Model

In [1]:
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
import datasets
import torch
from architectures import *
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint

%config Completer.use_jedi = False

In [2]:
dataset = datasets.SpecShadOcc("SpecShadOcc/")
test_length = 20
train_dataset,test_dataset = torch.utils.data.random_split(dataset, [len(dataset)-test_length,test_length])

In [3]:
comet_logger = CometLogger(
    api_key="tMEjeyq5M7v1IPRCvS5fyGyuo",
    workspace="semjon", # Optional
    project_name="deep-sequences", # Optional
    # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
    save_dir='./test',
    experiment_name="specshadocc" # Optional
)

train_loader = DataLoader(train_dataset, batch_size=9, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=10)


agg_params = { 
    "num_heads":8,
    "dim_feedforward":128,
    "num_layers":3
}
net = DeepAggNet(planes=16,agg_block="Attention",agg_params=agg_params)
trainer = pl.Trainer(gpus=1,logger=comet_logger, )

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [ ]:
trainer.fit(net, train_loader, val_dataloaders=test_loader)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/semjon/deep-sequences/7281fe3a16a74af3ba4b1903652af4c0


  | Name       | Type                 | Params
----------------------------------------------------
0 | input      | Conv2d               | 448   
1 | output     | Conv2d               | 435   
2 | downsample | Sequential           | 23.1 K
3 | upsample   | Sequential           | 23.1 K
4 | encoder    | Sequential           | 738 K 
5 | agg        | AttentionAggregation | 100 K 
6 | decoder    | Sequential           | 738 K 
7 | smooth     | Sequential           | 32.5 K
----------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.628     Total estimated model params size (MB)


Epoch 0:   0%|                                                                                  | 0/11 [00:00<?, ?it/s]

C:\Users\simon\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


Epoch 0:  82%|████████████████████████████████████████▉         | 9/11 [00:05<00:01,  1.64it/s, loss=0.223, v_num=f4c0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  91%|████████████████████████████████████████████▌    | 10/11 [00:05<00:00,  1.85it/s, loss=0.224, v_num=f4c0]
Validating: 0it [00:00, ?it/s]
Epoch 2:  91%|████████████████████████████████████████████▌    | 10/11 [00:05<00:00,  1.80it/s, loss=0.194, v_num=f4c0]
Validating: 0it [00:00, ?it/s]
Epoch 3:  91%|█████████████████████████████████████████████▍    | 10/11 [00:05<00:00,  1.81it/s, loss=0.14, v_num=f4c0]
Validating: 0it [00:00, ?it/s]
Epoch 4:  91%|████████████████████████████████████████████▌    | 10/11 [00:05<00:00,  1.81it/s, loss=0.105, v_num=f4c0]
Validating: 0it [00:00, ?it/s]
Epoch 5:  91%|███████████████████████████████████████████▋    | 10/11 [00:05<00:00,  1.79it/s, loss=0.0827, v_num=f4c0]
Validating: 0it [00:00, ?it/s]
Epoch 6:  91%|███████████████████████████████████████████▋    | 10/11 [00:05<00:00,  1.78it/s,